<a href="https://colab.research.google.com/github/preethi9999/info5731_spring2021/blob/main/somaraju_preethi_inclass8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [1]:
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp37-none-any.whl size=37070 sha256=5abbe8684e140828a3e34aca33f93b145d18e70b85e4a94ba4f2950409853269
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [6]:
import pytreebank
import sys
import os
path = os.path.join(sys.path[0],'sst_{}.txt')
dataset = pytreebank.load_sst('./cleaned_data')
for category in ['train', 'test']:
    with open(path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("__label__{}\t{}\n".format(
                item.to_labeled_lines()[0][0] + 1,
                item.to_labeled_lines()[0][1]
            ))

print(len(dataset['train']))

8544


In [10]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import TextBlob


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
in_data = pd.read_csv("/content/input_8.csv")
in_data = in_data[['Abstract']]
in_data['Abstract'] = in_data['Abstract'].str.replace('[^\w\s]','')
in_data['Abstract'] = in_data['Abstract'].apply(lambda x: " ".join(x.lower() for x in x.split()))
in_data['Abstract'] = in_data['Abstract'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
in_data['Abstract'] = in_data['Abstract'].apply(lambda x: nltk.word_tokenize(x))
data_freq = (in_data['Abstract']).apply(lambda x: pd.value_counts(x)).sum(axis = 0).reset_index()
data_freq.columns = ['words', 'tf value']
data_freq['polarity score'] = data_freq['words'].apply(lambda x: TextBlob(x).sentiment.polarity)
senti_terms = data_freq.loc[data_freq['polarity score'] != 0].sort_values(by='tf value', ascending=False)
senti_terms = senti_terms.reset_index(drop=True)
print('Ranking sentiment based on frequency:')
senti_terms

Ranking sentiment based on frequency:


,words,tf value,polarity score
0,natural,71.0,0.100000
1,linguistic,10.0,0.100000
2,new,8.0,0.136364
3,many,7.0,0.500000
4,effective,7.0,0.600000
...,...,...,...
85,cultural,1.0,0.100000
86,conventional,1.0,-0.142857
87,contemporary,1.0,0.166667
88,fit,1.0,0.400000


(10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers.

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [19]:
import pandas as pd
df=pd.read_csv("/content/sample_data/cleaned_data.csv",usecols=["clean text","sentiment"])
df=df.dropna().reset_index()

In [20]:
df

,index,clean text,sentiment
0,0,introduction statistical natural language proc...,positive
1,1,the paper summarizes essential property docume...,positive
2,2,abstract language way communicating word langu...,positive
3,3,we report experiment use standard natural lang...,positive
4,4,paper describe simple rule based approach auto...,positive
...,...,...,...
92,95,bibliography,positive
93,96,applying natural language processing technique...,positive
94,97,example natural language chinese english itali...,positive
95,98,a number powerful modelling technique develope...,positive


In [51]:
from sklearn.metrics import f1_score, accuracy_score
textblob_data = pd.read_csv("/content/sample_data/final_input.csv")
textblob_data['polarity'] = textblob_data['Abstract'].apply(lambda x: TextBlob(x).sentiment.polarity)
textblob_data['predicted_sentiment'] = pd.cut(textblob_data['polarity'], bins=4, labels=[1, 2, 3, 4])
def sentiment(x):
    if x in [1, 2]:
        return 'Negative'
    if x == 3:
        return 'Neutral'
    if x == 4:
        return 'Positive'
textblob_data['predicted_sentiment'] = textblob_data['predicted_sentiment'].apply(lambda x: sentiment(x))
textblob_data
print(textblob_data[['doc_id','sentiment', 'predicted_sentiment']].head())
textblob_accuracy = accuracy_score(textblob_data['sentiment'], textblob_data['predicted_sentiment'])*100
textblob_f1 = f1_score(textblob_data['sentiment'], textblob_data['predicted_sentiment'], average='macro')
print('accuracy using textBlob:', textblob_accuracy)
print('f1score using textBlob:', textblob_f1)

   doc_id sentiment predicted_sentiment
0       0  positive             Neutral
1       1  positive             Neutral
2       2  positive             Neutral
3       3  positive             Neutral
4       4  positive             Neutral
accuracy using textBlob: 0.0
f1score using textBlob: 0.0


In [52]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()
vader_data = pd.read_csv("/content/sample_data/final_input.csv")
vader_data['polarity'] = textblob_data['Abstract'].apply(lambda x: vader.polarity_scores(x)['compound'])
vader_data['predicted_sentiment'] = pd.cut(vader_data['polarity'], bins=4, labels=[1, 2, 3, 4])
vader_data['predicted_sentiment'] = vader_data['predicted_sentiment'].apply(lambda x: sentiment(x))
print(vader_data[['doc_id', 'sentiment', 'predicted_sentiment']].head())
vader_accuracy = accuracy_score(vader_data['sentiment'], vader_data['predicted_sentiment'])*100
vader_f1 = f1_score(vader_data['sentiment'], vader_data['predicted_sentiment'], average='macro')
print('accuracy using vader sentiment analysis:', vader_accuracy)
print('f1score using vader sentiment analysis:', vader_f1)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
   doc_id sentiment predicted_sentiment
0       0  positive             Neutral
1       1  positive            Positive
2       2  positive            Positive
3       3  positive            Positive
4       4  positive            Negative
accuracy using vader sentiment analysis: 0.0
f1score using vader sentiment analysis: 0.0


In [54]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

svm_input = pd.read_csv("/content/sample_data/final_input.csv")
train, test = sklearn.model_selection.train_test_split(svm_input, train_size=0.6, test_size=0.1)
mypip = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=100, 
                                           learning_rate='optimal', tol=None))])

svm = mypip.fit(train['Abstract'], train['sentiment'])
test['predicted_sentiment'] = svm.predict(test['Abstract'])
print(test[['doc_id', 'sentiment', 'predicted_sentiment']].head())

svm_accuracy = accuracy_score(test['sentiment'], test['predicted_sentiment'])*100
svm_f1 = f1_score(test['sentiment'], test['predicted_sentiment'], average='macro')

print('accuracy using TFIDF-based SVM:', svm_accuracy)
print('f1score using TFIDF-based SVM :', svm_f1)

    doc_id sentiment predicted_sentiment
72      72   neutral            positive
70      70  positive            positive
28      28  negative            positive
7        7  positive            positive
76      76  positive            positive
accuracy using TFIDF-based SVM: 70.0
f1score using TFIDF-based SVM : 0.27450980392156865


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Analysis

In [53]:
'''
In the above sentiment analysis is performed using below packages

Textblob : It is a python library used for textual processing. It provides a simple API for diving into common natural language processing (NLP) tasks 
such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.
Textblob returns two properties of input sentence that is polarity and subjectivity.

VADER : It is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.
Vader uses a combination of A sentiment lexicon is a list of lexical features (e.g., words) which are generally labeled according to their semantic orientation as either positive or negative. 
Vader not only tells about the Positivity and Negativity score but also tells us about how positive or negative a sentiment is.

SVM : Support vector machine (SVM) is a learning technique that performs well on sentiment classification. Tf-IDF lets us know know the importance of word in a document as it relates to frequency.
SVM is a supervised machine learning algorithm that can be used for both classification or regression challenges.

we can measure the accuracy of model using F1 score/Fscore. A good F1 score means it has low false positives and false negatives and it is near to expected result, where as 
bad f1 score indicates more of false positives and negatives. With good fscore we can identify the real threats.

As per my analysis , we can see that F1 score using SVM is good and accurate compared to other two models.
Out of three models SVM is good for sentiment analysis as f1 score and accuracy is low for Text blob and vader methods.

'''

